Preparación de Datos

Importar Librerías Necesarias

In [27]:
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split

Descargar Recursos de NLTK

In [28]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

Cargar el Conjunto de Datos

In [29]:
# Cargarmos el conjunto de datos 20 Newsgroups de la siguiente manera
newsgroups = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))
texts = newsgroups.data
labels = newsgroups.target

Preprocesamiento de Texto

In [30]:
# Definimos lad funcones de preprocesamiento
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    text = text.lower()
    # Eliminar caracteres especiales y números para quedarnos solamente con caracteres permitidos
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Tokenizar
    tokens = text.split()
    # lematizamos y normalizamos
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    text = ' '.join(tokens)
    return text

texts_clean = [preprocess_text(text) for text in texts]

Preprocesamiento de Texto

In [31]:
X_train, X_temp, y_train, y_temp = train_test_split(texts_clean, labels, test_size=0.3, random_state=42, stratify=labels)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

Construcción de Embeddings

Entrenar Word2Vec en el Conjunto de Entrenamiento

In [32]:
from gensim.models import Word2Vec

train_tokens = [text.split() for text in X_train]

# Entrenamos mpdelo Word2Vec, el cual tomará los tokens de las oraciones y
# creará vectores de tamaño 100, con una ventana de contexto de 5 palabras,
# ignorando palabras que aparezcan menos de 2 veces.
w2v_model = Word2Vec(sentences=train_tokens, vector_size=100, window=5, min_count=2, workers=4)

Crear Representaciones de Documentos

In [33]:
# Función para obtener el vector promedio de un documento
def document_vector(doc):
    # Conservamos solo las palabras que estén en el vocabulario del modelo Word2Vec
    doc = [word for word in doc.split() if word in w2v_model.wv.index_to_key]
    if len(doc) == 0:
        return np.zeros(w2v_model.vector_size)
    else:
        return np.mean(w2v_model.wv[doc], axis=0)

# Obtenemos los vectores de todows los documentos
X_train_vect = np.array([document_vector(doc) for doc in X_train])
X_val_vect = np.array([document_vector(doc) for doc in X_val])
X_test_vect = np.array([document_vector(doc) for doc in X_test])

Implementación del Modelo

In [34]:
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

Crear un Pipeline de Preprocesamiento y Modelo

In [35]:
# Estandarizamos los datos y aplicamos el clasificador
model = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', SGDClassifier(loss='log_loss', penalty='l2', max_iter=1000, random_state=42))
])


Entrenamiento del Modelo

Entrenar el Modelo con el Conjunto de Entrenamiento

In [36]:
model.fit(X_train_vect, y_train)


Pipeline(steps=[('scaler', StandardScaler()),
                ('classifier',
                 SGDClassifier(loss='log_loss', random_state=42))])